In [95]:
import nemo
import copy
import tqdm
import timeit
import pathlib
import attrdict
import numpy as np

from ruamel import yaml
from Bio import pairwise2
from nemo.collections import asr as nemo_asr
from nemo.collections.asr.helpers import post_process_predictions, post_process_transcripts, word_error_rate

In [24]:
runner = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU
)

In [25]:
MODEL_CONFIG = 'examples/asr/configs/quartznet15x5.yaml'
yaml_loader = yaml.YAML(typ="safe")
with open(MODEL_CONFIG) as f:
    config = attrdict.AttrDict(yaml_loader.load(f))
labels = list(config.labels)
str(labels)

'[\' \', \'a\', \'b\', \'c\', \'d\', \'e\', \'f\', \'g\', \'h\', \'i\', \'j\', \'k\', \'l\', \'m\', \'n\', \'o\', \'p\', \'q\', \'r\', \'s\', \'t\', \'u\', \'v\', \'w\', \'x\', \'y\', \'z\', "\'"]'

In [91]:
EVAL_DATASETS = [
    '/home/stanislavv/data/nemo-librispeech/train_clean_100.json',
    '/home/stanislavv/data/nemo-librispeech/train_clean_360.json',
    '/home/stanislavv/data/nemo-librispeech/train_other_500.json',
]
eval_dl_params = copy.deepcopy(config.AudioToTextDataLayer)
eval_dl_params.update(config.AudioToTextDataLayer["eval"])
del eval_dl_params["train"]
del eval_dl_params["eval"]
data_layer = nemo_asr.AudioToTextDataLayer(
    manifest_filepath=','.join(EVAL_DATASETS),
    sample_rate=config.sample_rate,
    labels=config.labels,
    batch_size=32,
    **eval_dl_params,
)

[NeMo I 2020-03-04 14:19:18 collections:138] Dataset loaded with 281241 files totalling 961.05 hours
[NeMo I 2020-03-04 14:19:18 collections:139] 0 files were filtered totalling 0.00 hours


In [92]:
data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor(
    sample_rate=config.sample_rate, **config.AudioToMelSpectrogramPreprocessor
)
jasper_encoder = nemo_asr.JasperEncoder(
    feat_in=config.AudioToMelSpectrogramPreprocessor["features"], **config.JasperEncoder
)
jasper_decoder = nemo_asr.JasperDecoderForCTC(
    feat_in=config.JasperEncoder["jasper"][-1]["filters"], num_classes=len(config.labels)
)
greedy_decoder = nemo_asr.GreedyCTCDecoder()

[NeMo I 2020-03-04 14:19:23 features:144] PADDING: 16
[NeMo I 2020-03-04 14:19:23 features:152] STFT using conv


In [93]:
audio_signal_e1, a_sig_length_e1, transcript_e1, transcript_len_e1 = data_layer()
processed_signal_e1, p_length_e1 = data_preprocessor(input_signal=audio_signal_e1, length=a_sig_length_e1)
encoded_e1, encoded_len_e1 = jasper_encoder(audio_signal=processed_signal_e1, length=p_length_e1)
log_probs_e1 = jasper_decoder(encoder_output=encoded_e1)
predictions_e1 = greedy_decoder(log_probs=log_probs_e1)

In [94]:
eval_tensors = [log_probs_e1, predictions_e1, transcript_e1, transcript_len_e1, encoded_len_e1, p_length_e1]
load_dir = '/home/stanislavv/data/nemo-quartznet-15x5'
evaluated_tensors = runner.infer(tensors=eval_tensors, checkpoint_dir=load_dir)

[NeMo I 2020-03-04 14:19:42 actions:1456] Restoring JasperEncoder from /home/stanislavv/data/nemo-quartznet-15x5/JasperEncoder-STEP-406556.pt
[NeMo I 2020-03-04 14:19:42 actions:1456] Restoring JasperDecoderForCTC from /home/stanislavv/data/nemo-quartznet-15x5/JasperDecoderForCTC-STEP-406556.pt
[NeMo I 2020-03-04 14:19:42 actions:729] Evaluating batch 0 out of 8789
[NeMo I 2020-03-04 14:25:19 actions:729] Evaluating batch 878 out of 8789
[NeMo I 2020-03-04 14:30:57 actions:729] Evaluating batch 1756 out of 8789
[NeMo I 2020-03-04 14:36:21 actions:729] Evaluating batch 2634 out of 8789
[NeMo I 2020-03-04 14:41:41 actions:729] Evaluating batch 3512 out of 8789
[NeMo I 2020-03-04 14:47:15 actions:729] Evaluating batch 4390 out of 8789
[NeMo I 2020-03-04 14:52:46 actions:729] Evaluating batch 5268 out of 8789
[NeMo I 2020-03-04 14:58:20 actions:729] Evaluating batch 6146 out of 8789
[NeMo I 2020-03-04 15:03:56 actions:729] Evaluating batch 7024 out of 8789
[NeMo I 2020-03-04 15:09:19 actio

In [30]:
references = post_process_transcripts(evaluated_tensors[2], evaluated_tensors[3], config.labels)
greedy_hypotheses = post_process_predictions(evaluated_tensors[1], config.labels)
word_error_rate(greedy_hypotheses, references)

0.03860152200286754

In [131]:
ctc_tokens = evaluated_tensors[1]
len(ctc_tokens)

8789

In [139]:
i = 0
for ctc_tokens1 in ctc_tokens:
    if not (ctc_tokens1[:, 0] == 28).all():
        i += (ctc_tokens1[:, 0] == 28).sum()

print(i)

tensor(44138)


In [33]:
text = evaluated_tensors[2][0]
text_len = evaluated_tensors[3][0]
ctc_tokens = evaluated_tensors[1][0]
ctc_logprobs = evaluated_tensors[0][0]
ctc_len = evaluated_tensors[4][0]
mel_len = evaluated_tensors[-1][0]

In [34]:
text1 = text[0].numpy()
text_len1 = text_len[0].numpy().item()
ctc_tokens1 = ctc_tokens[0].numpy()
ctc_logprobs1 = ctc_logprobs[0].numpy()
ctc_len1 = ctc_len[0].numpy().item()
mel_len1 = mel_len[0].numpy().item()

In [35]:
text1 = list(text1[:text_len1])
ctc_tokens1 = list(ctc_tokens1[:ctc_len1])
ctc_logprobs1 = ctc_logprobs1[:ctc_len1]
len(text1), len(ctc_tokens1), ctc_logprobs1.shape

(161, 585, (585, 29))

In [158]:
class Processer:
    def __init__(self, labels):
        labels = labels + ['<BLANK>']
        self.space_id = labels.index(' ')
        self.blank_id = len(labels) - 1
        self.labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    
    def bound_text(self, tokens):
        return [self.space_id] + tokens + [self.space_id]
    
    def bound_ctc(self, tokens, logprobs):
        tokens = [self.space_id, self.blank_id] + tokens + [self.blank_id, self.space_id]
        
        logprobs = np.lib.pad(logprobs, ((2, 2), (0, 0)), 'edge')

        def swap(col, a, b):
            x[col][a], x[col][b] = x[col][b], x[col][a]
        
        first_token, last_token = tokens[2], tokens[-3]
        swap(0, first_token, self.space_id)
        swap(1, first_token, self.blank_id)
        swap(-1, last_token, self.space_id)
        swap(-2, last_token, self.blank_id)

        return tokens, logprobs
    
    def merge(self, tokens):
        output_tokens = []
        output_cnts = []
        cnt = 0
        for i in range(len(tokens)):
            if i != 0 and (tokens[i - 1] != tokens[i]):
                output_tokens.append(tokens[i - 1])
                output_cnts.append(cnt)

                cnt = 0

            cnt += 1

        output_tokens.append(tokens[-1])
        output_cnts.append(cnt)
        
        assert sum(output_cnts) == len(tokens), f'SUM_CHECK {sum(output_cnts)} vs {len(tokens)}'

        return output_tokens, output_cnts
    
    def merge_with_blanks(self, tokens, cnts, logprobs=None):
        def choose_sep(l, r, a, b):
            # `tokens[l] == a and tokens[r] == b`.
            sum_a, sum_b = logprobs[l, a], logprobs[l + 1:r + 1, b].sum()
            best_sum, best_sep = sum_a + sum_b, 0
            for sep in range(1, r - l):
                sum_a += logprobs[l + sep, a]
                sum_b -= logprobs[l + sep, b]
                if sum_a + sum_b > best_sum:
                    best_sum, best_sep = sum_a + sum_b, sep

            return best_sep
        
        output_tokens = []
        output_durs = []
        blank_cnt = 0
        total_cnt = 0
        for token, cnt in zip(tokens, cnts):
            total_cnt += cnt
            if token == self.blank_id:
                blank_cnt += cnt
                continue
            
            output_tokens.append(token)
            
            if logprobs is None:
                # Half half.
                left_cnt = blank_cnt // 2
            else:
                # Clever sep choice based on sum of log probs.
                left_cnt = choose_sep(
                    l=total_cnt - cnt - blank_cnt - 1,
                    r=total_cnt - cnt,
                    a=output_tokens[-1],
                    b=token,
                )
            right_cnt = blank_cnt - left_cnt
            blank_cnt = 0
            
            if left_cnt:
                output_durs[-1] += left_cnt
            output_durs.append(cnt + right_cnt)
        
        output_durs[-1] += blank_cnt

        assert sum(output_durs) == sum(cnts), f'SUM_CHECK {sum(output_durs)} vs {sum(cnts)}'

        return output_tokens, output_durs
    
    def align(self, output_tokens, gt_text):
        def make_str(tokens):
            return ''.join(self.labels_map[c] for c in tokens)
        
        s = make_str(output_tokens)
        t = make_str(gt_text)
        alignmet = pairwise2.align.globalxx(s, t)[0]
        sa, ta, *_ = alignmet
        return sa, ta
    
    def generate(self, gt_text, alignment, durs):
        output_tokens = []
        output_cnts = []
        si, ti = 0, 0    
        for sc, tc in zip(*alignment):
            if sc == '-':
                output_tokens.append(self.blank_id)
                output_cnts.append(2 * durs[si])
                si += 1
            elif tc == '-':
                output_tokens.append(gt_text[ti])
                output_cnts.append(0)
                ti += 1
            else:
                output_tokens.append(gt_text[ti])
                output_cnts.append(2 * durs[si])
                si += 1
                ti += 1

        assert sum(output_cnts) == 2 * sum(durs)
        
        return output_tokens, output_cnts

    def __call__(self, text, ctc_tokens, ctc_logprobs, mel_len):
        # This adds +2 tokens.
        text = self.bound_text(text)
        # This add +4 tokens, 2 of them are blank.
        ctc_tokens, ctc_logprobs = self.bound_ctc(ctc_tokens, ctc_logprobs)

        ctc_tokens, ctc_cnts = self.merge(ctc_tokens)
        ctc_tokens, ctc_durs = self.merge_with_blanks(ctc_tokens, ctc_cnts, ctc_logprobs)
        
        alignment = self.align(text, ctc_tokens)
        tokens, cnts = self.generate(text, alignment, ctc_durs)
        tokens, durs = self.merge_with_blanks(tokens, cnts)
        assert tokens == text, 'EXACT_TOKENS_MATCH_CHECK'

        def adjust(start, direction, value):
            i = start
            while value != 0:
                dur = durs[i]
                
                if value < 0:
                    durs[i] = dur - value
                else:
                    durs[i] = max(dur - value, 0)
                
                value -= dur - durs[i]
                i += direction

        adjust(0, 1, 4)
        adjust(-1, -1, sum(durs) - mel_len)  # Including 4 suffix bound tokens.
        assert durs[0] >= 0, f'{durs[0]}'
        assert durs[-1] >= 0, f'{durs[-1]}'
        
        durs = np.array(durs, dtype=np.long)
        assert durs.shape[0] == len(text), f'LEN_CHECK {durs.shape[0]} vs {len(text)}'
        assert np.sum(durs) == mel_len, f'SUM_CHECK {np.sum(durs)} vs {mel_len}'

        return durs

processer = Processer(labels)
durs1 = processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)
durs1

array([34, 18,  6,  2, 16, 16, 10,  4,  6, 56,  6,  4,  2, 16, 10,  8,  4,
        6, 20,  6,  2, 10,  8,  6,  2, 10,  6, 92, 12,  2,  4,  2, 14,  2,
        4,  4,  2,  8,  6,  4,  4, 12,  6,  2,  4,  2,  2,  4,  4,  6,  2,
        2,  2,  6,  4,  4,  6,  8,  6,  2,  4,  4,  2,  4,  2,  4,  4,  2,
        4,  4, 18,  2,  4,  8,  6, 22,  8,  4,  4,  6, 14,  8,  8,  2,  6,
        4, 10,  4,  4,  6,  6,  6,  6,  4, 16,  6,  8,  2,  8, 18,  2,  2,
        2,  2, 12,  4,  6, 10,  2,  8,  6,  2,  6,  4,  2, 12,  4,  6,  2,
        2, 82,  8,  6,  2,  2,  8, 18,  4,  8, 10, 28,  8,  2,  4, 14,  4,
        8,  4,  6,  6,  2,  4,  8,  4, 10,  6,  8,  2,  4, 10,  2, 10,  4,
       18,  4,  6,  6,  4,  4,  6,  4,  4, 14, 12, 14, 10,  8,  6,  2, 10,
        2,  2,  6,  2,  2,  2,  2,  6,  4,  2,  8,  4,  4, 10,  4,  4,  6,
        4, 49])

In [89]:
f = lambda: processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)
n, m = 10000, 280000
(timeit.timeit(f, number=n) / n) * m

766.232532940805

In [159]:
# NOTE: This is going pretty fast.
durs_dir = pathlib.Path('/home/stanislavv/data/nemo-librispeech/durs')
durs_dir.mkdir(exist_ok=True)
k = -1

for batch in tqdm.tqdm(zip(*evaluated_tensors), total=8789):
    text = batch[2].numpy()
    text_len = batch[3].numpy()
    ctc_tokens = batch[1].numpy()
    ctc_logprobs = batch[0].numpy()
    ctc_len = batch[4].numpy()
    mel_len = batch[-1].numpy()

    for text1, text_len1, ctc_tokens1, ctc_logprobs1, ctc_len1, mel_len1 in zip(
        text, text_len, ctc_tokens, ctc_logprobs, ctc_len, mel_len
    ):
        text1 = list(text1[:text_len1])
        ctc_tokens1 = list(ctc_tokens1[:ctc_len1])
        ctc_logprobs1 = ctc_logprobs1[:ctc_len1]
        mel_len1 = mel_len1.item()

        durs1 = processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)

        k += 1
        np.save(durs_dir / f'{k}.npy', durs1, allow_pickle=False)





  0%|          | 0/8789 [00:00<?, ?it/s]



  0%|          | 1/8789 [00:00<21:12,  6.91it/s]



  0%|          | 2/8789 [00:00<19:33,  7.49it/s]



  0%|          | 3/8789 [00:00<19:32,  7.49it/s]



  0%|          | 4/8789 [00:00<19:03,  7.68it/s]



  0%|          | 5/8789 [00:00<19:01,  7.70it/s]



  0%|          | 6/8789 [00:00<18:43,  7.82it/s]



  0%|          | 7/8789 [00:00<19:10,  7.63it/s]



  0%|          | 8/8789 [00:01<18:55,  7.73it/s]



  0%|          | 9/8789 [00:01<20:33,  7.12it/s]



  0%|          | 10/8789 [00:01<20:56,  6.99it/s]



  0%|          | 11/8789 [00:01<21:54,  6.68it/s]



  0%|          | 12/8789 [00:01<20:52,  7.01it/s]



  0%|          | 13/8789 [00:01<20:49,  7.03it/s]



  0%|          | 14/8789 [00:01<19:22,  7.55it/s]



  0%|          | 15/8789 [00:01<18:28,  7.91it/s]



  0%|          | 16/8789 [00:02<18:55,  7.73it/s]



  0%|          | 17/8789 [00:02<19:32,  7.48it/s]



  0%|          | 18/8789 [00:02<19:11,  7.62it/s]



  0%|  

  3%|▎         | 255/8789 [00:35<22:06,  6.43it/s]



  3%|▎         | 256/8789 [00:36<23:00,  6.18it/s]



  3%|▎         | 257/8789 [00:36<22:51,  6.22it/s]



  3%|▎         | 258/8789 [00:36<20:42,  6.87it/s]



  3%|▎         | 259/8789 [00:36<20:57,  6.78it/s]



  3%|▎         | 261/8789 [00:36<18:56,  7.50it/s]



  3%|▎         | 262/8789 [00:36<18:12,  7.81it/s]



  3%|▎         | 263/8789 [00:37<18:19,  7.75it/s]



  3%|▎         | 264/8789 [00:37<18:14,  7.79it/s]



  3%|▎         | 265/8789 [00:37<18:20,  7.75it/s]



  3%|▎         | 266/8789 [00:37<17:20,  8.19it/s]



  3%|▎         | 267/8789 [00:37<17:47,  7.99it/s]



  3%|▎         | 268/8789 [00:37<17:34,  8.08it/s]



  3%|▎         | 270/8789 [00:37<17:01,  8.34it/s]



  3%|▎         | 271/8789 [00:37<17:07,  8.29it/s]



  3%|▎         | 272/8789 [00:38<17:47,  7.98it/s]



  3%|▎         | 273/8789 [00:38<17:36,  8.06it/s]



  3%|▎         | 274/8789 [00:38<19:44,  7.19it/s]



  3%|▎         | 275/8789 [0

  6%|▌         | 513/8789 [01:10<18:39,  7.39it/s]



  6%|▌         | 514/8789 [01:10<19:28,  7.08it/s]



  6%|▌         | 515/8789 [01:10<19:47,  6.97it/s]



  6%|▌         | 516/8789 [01:10<19:59,  6.89it/s]



  6%|▌         | 517/8789 [01:10<19:15,  7.16it/s]



  6%|▌         | 518/8789 [01:11<20:12,  6.82it/s]



  6%|▌         | 519/8789 [01:11<20:44,  6.64it/s]



  6%|▌         | 520/8789 [01:11<21:13,  6.49it/s]



  6%|▌         | 521/8789 [01:11<19:54,  6.92it/s]



  6%|▌         | 522/8789 [01:11<19:33,  7.05it/s]



  6%|▌         | 523/8789 [01:11<19:38,  7.01it/s]



  6%|▌         | 524/8789 [01:11<20:25,  6.75it/s]



  6%|▌         | 525/8789 [01:12<20:29,  6.72it/s]



  6%|▌         | 526/8789 [01:12<20:35,  6.69it/s]



  6%|▌         | 527/8789 [01:12<21:04,  6.53it/s]



  6%|▌         | 528/8789 [01:12<23:49,  5.78it/s]



  6%|▌         | 529/8789 [01:12<23:12,  5.93it/s]



  6%|▌         | 530/8789 [01:12<23:20,  5.90it/s]



  6%|▌         | 531/8789 [0

  9%|▊         | 767/8789 [01:45<18:28,  7.23it/s]



  9%|▊         | 768/8789 [01:45<17:40,  7.56it/s]



  9%|▊         | 769/8789 [01:45<17:14,  7.75it/s]



  9%|▉         | 770/8789 [01:45<17:11,  7.77it/s]



  9%|▉         | 771/8789 [01:46<18:43,  7.14it/s]



  9%|▉         | 772/8789 [01:46<18:38,  7.17it/s]



  9%|▉         | 773/8789 [01:46<19:14,  6.94it/s]



  9%|▉         | 774/8789 [01:46<18:05,  7.38it/s]



  9%|▉         | 775/8789 [01:46<16:45,  7.97it/s]



  9%|▉         | 776/8789 [01:46<16:25,  8.13it/s]



  9%|▉         | 777/8789 [01:46<16:52,  7.92it/s]



  9%|▉         | 778/8789 [01:47<17:18,  7.72it/s]



  9%|▉         | 779/8789 [01:47<18:07,  7.37it/s]



  9%|▉         | 780/8789 [01:47<18:48,  7.10it/s]



  9%|▉         | 781/8789 [01:47<19:18,  6.91it/s]



  9%|▉         | 782/8789 [01:47<19:44,  6.76it/s]



  9%|▉         | 783/8789 [01:47<21:53,  6.10it/s]



  9%|▉         | 784/8789 [01:48<21:46,  6.13it/s]



  9%|▉         | 785/8789 [0

 12%|█▏        | 1023/8789 [02:21<21:14,  6.09it/s]



 12%|█▏        | 1024/8789 [02:21<20:43,  6.24it/s]



 12%|█▏        | 1025/8789 [02:21<20:47,  6.23it/s]



 12%|█▏        | 1026/8789 [02:21<20:22,  6.35it/s]



 12%|█▏        | 1027/8789 [02:21<19:28,  6.64it/s]



 12%|█▏        | 1028/8789 [02:22<19:26,  6.65it/s]



 12%|█▏        | 1029/8789 [02:22<20:15,  6.38it/s]



 12%|█▏        | 1030/8789 [02:22<18:49,  6.87it/s]



 12%|█▏        | 1032/8789 [02:22<16:33,  7.81it/s]



 12%|█▏        | 1033/8789 [02:22<17:12,  7.52it/s]



 12%|█▏        | 1034/8789 [02:22<17:33,  7.36it/s]



 12%|█▏        | 1035/8789 [02:22<17:42,  7.30it/s]



 12%|█▏        | 1036/8789 [02:23<16:47,  7.69it/s]



 12%|█▏        | 1037/8789 [02:23<16:52,  7.65it/s]



 12%|█▏        | 1038/8789 [02:23<16:30,  7.83it/s]



 12%|█▏        | 1039/8789 [02:23<16:40,  7.75it/s]



 12%|█▏        | 1040/8789 [02:23<17:29,  7.38it/s]



 12%|█▏        | 1041/8789 [02:23<17:51,  7.23it/s]



 12%|█▏   

...

 86%|████████▋ | 7595/8789 [16:34<02:28,  8.03it/s]



 86%|████████▋ | 7596/8789 [16:34<02:30,  7.94it/s]



 86%|████████▋ | 7597/8789 [16:34<02:30,  7.93it/s]



 86%|████████▋ | 7598/8789 [16:34<02:27,  8.06it/s]



 86%|████████▋ | 7599/8789 [16:35<02:33,  7.77it/s]



 86%|████████▋ | 7600/8789 [16:35<02:32,  7.78it/s]



 86%|████████▋ | 7601/8789 [16:35<02:27,  8.03it/s]



 86%|████████▋ | 7602/8789 [16:35<02:27,  8.03it/s]



 87%|████████▋ | 7603/8789 [16:35<02:24,  8.21it/s]



 87%|████████▋ | 7605/8789 [16:35<02:14,  8.77it/s]



 87%|████████▋ | 7606/8789 [16:35<02:11,  9.00it/s]



 87%|████████▋ | 7607/8789 [16:36<02:33,  7.70it/s]



 87%|████████▋ | 7608/8789 [16:36<02:41,  7.31it/s]



 87%|████████▋ | 7609/8789 [16:36<02:38,  7.46it/s]



 87%|████████▋ | 7610/8789 [16:36<02:45,  7.10it/s]



 87%|████████▋ | 7611/8789 [16:36<02:57,  6.62it/s]



 87%|████████▋ | 7612/8789 [16:36<03:05,  6.36it/s]



 87%|████████▋ | 7613/8789 [16:36<02:58,  6.58it/s]



 87%|█████

 90%|████████▉ | 7878/8789 [17:09<01:41,  8.97it/s]



 90%|████████▉ | 7880/8789 [17:09<01:40,  9.05it/s]



 90%|████████▉ | 7881/8789 [17:10<01:45,  8.61it/s]



 90%|████████▉ | 7882/8789 [17:10<01:45,  8.59it/s]



 90%|████████▉ | 7883/8789 [17:10<02:02,  7.41it/s]



 90%|████████▉ | 7885/8789 [17:10<01:50,  8.20it/s]



 90%|████████▉ | 7887/8789 [17:10<01:42,  8.78it/s]



 90%|████████▉ | 7888/8789 [17:10<01:45,  8.54it/s]



 90%|████████▉ | 7889/8789 [17:10<01:57,  7.66it/s]



 90%|████████▉ | 7890/8789 [17:11<02:03,  7.28it/s]



 90%|████████▉ | 7891/8789 [17:11<02:01,  7.38it/s]



 90%|████████▉ | 7892/8789 [17:11<01:58,  7.57it/s]



 90%|████████▉ | 7893/8789 [17:11<01:56,  7.68it/s]



 90%|████████▉ | 7895/8789 [17:11<01:52,  7.93it/s]



 90%|████████▉ | 7896/8789 [17:11<01:53,  7.84it/s]



 90%|████████▉ | 7897/8789 [17:12<01:52,  7.96it/s]



 90%|████████▉ | 7898/8789 [17:12<02:00,  7.37it/s]



 90%|████████▉ | 7899/8789 [17:12<02:02,  7.26it/s]



 90%|█████

 93%|█████████▎| 8157/8789 [17:44<01:15,  8.40it/s]



 93%|█████████▎| 8158/8789 [17:44<01:18,  8.06it/s]



 93%|█████████▎| 8159/8789 [17:44<01:17,  8.16it/s]



 93%|█████████▎| 8160/8789 [17:45<01:23,  7.52it/s]



 93%|█████████▎| 8161/8789 [17:45<01:28,  7.10it/s]



 93%|█████████▎| 8162/8789 [17:45<01:26,  7.28it/s]



 93%|█████████▎| 8163/8789 [17:45<01:25,  7.28it/s]



 93%|█████████▎| 8164/8789 [17:45<01:38,  6.33it/s]



 93%|█████████▎| 8165/8789 [17:45<01:44,  5.99it/s]



 93%|█████████▎| 8166/8789 [17:45<01:35,  6.51it/s]



 93%|█████████▎| 8167/8789 [17:46<01:38,  6.29it/s]



 93%|█████████▎| 8168/8789 [17:46<01:31,  6.77it/s]



 93%|█████████▎| 8169/8789 [17:46<01:28,  6.97it/s]



 93%|█████████▎| 8170/8789 [17:46<01:25,  7.26it/s]



 93%|█████████▎| 8171/8789 [17:46<01:21,  7.55it/s]



 93%|█████████▎| 8172/8789 [17:46<01:20,  7.70it/s]



 93%|█████████▎| 8173/8789 [17:46<01:20,  7.68it/s]



 93%|█████████▎| 8174/8789 [17:47<01:19,  7.74it/s]



 93%|█████

 96%|█████████▌| 8435/8789 [18:18<00:43,  8.09it/s]



 96%|█████████▌| 8436/8789 [18:18<00:42,  8.31it/s]



 96%|█████████▌| 8437/8789 [18:18<00:42,  8.28it/s]



 96%|█████████▌| 8438/8789 [18:19<00:40,  8.61it/s]



 96%|█████████▌| 8439/8789 [18:19<00:40,  8.57it/s]



 96%|█████████▌| 8440/8789 [18:19<00:43,  7.94it/s]



 96%|█████████▌| 8441/8789 [18:19<00:45,  7.58it/s]



 96%|█████████▌| 8442/8789 [18:19<00:44,  7.73it/s]



 96%|█████████▌| 8443/8789 [18:19<00:46,  7.45it/s]



 96%|█████████▌| 8444/8789 [18:19<00:44,  7.72it/s]



 96%|█████████▌| 8446/8789 [18:19<00:40,  8.38it/s]



 96%|█████████▌| 8448/8789 [18:20<00:37,  8.99it/s]



 96%|█████████▌| 8450/8789 [18:20<00:35,  9.51it/s]



 96%|█████████▌| 8452/8789 [18:20<00:35,  9.60it/s]



 96%|█████████▌| 8453/8789 [18:20<00:36,  9.24it/s]



 96%|█████████▌| 8454/8789 [18:20<00:35,  9.34it/s]



 96%|█████████▌| 8455/8789 [18:20<00:36,  9.12it/s]



 96%|█████████▌| 8456/8789 [18:21<00:38,  8.57it/s]



 96%|█████

 99%|█████████▉| 8699/8789 [18:52<00:10,  8.97it/s]



 99%|█████████▉| 8700/8789 [18:53<00:10,  8.24it/s]



 99%|█████████▉| 8701/8789 [18:53<00:11,  7.73it/s]



 99%|█████████▉| 8702/8789 [18:53<00:10,  8.03it/s]



 99%|█████████▉| 8703/8789 [18:53<00:10,  8.18it/s]



 99%|█████████▉| 8704/8789 [18:53<00:11,  7.72it/s]



 99%|█████████▉| 8705/8789 [18:53<00:10,  7.94it/s]



 99%|█████████▉| 8706/8789 [18:53<00:10,  7.81it/s]



 99%|█████████▉| 8708/8789 [18:53<00:09,  8.67it/s]



 99%|█████████▉| 8710/8789 [18:54<00:08,  9.47it/s]



 99%|█████████▉| 8712/8789 [18:54<00:07, 10.00it/s]



 99%|█████████▉| 8714/8789 [18:54<00:08,  9.26it/s]



 99%|█████████▉| 8715/8789 [18:54<00:08,  8.67it/s]



 99%|█████████▉| 8716/8789 [18:54<00:09,  7.89it/s]



 99%|█████████▉| 8717/8789 [18:54<00:08,  8.03it/s]



 99%|█████████▉| 8718/8789 [18:55<00:08,  8.43it/s]



 99%|█████████▉| 8719/8789 [18:55<00:07,  8.82it/s]



 99%|█████████▉| 8720/8789 [18:55<00:07,  8.88it/s]



 99%|█████